In [60]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pathlib import Path

In [61]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [62]:
train_df.dtypes

Unnamed: 0                      int64
index                           int64
loan_amnt                     float64
int_rate                      float64
installment                   float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

ML role: PRIDICTS 
i)  Categories: to predict the components/category that the data set belongs to; 
ii) Continous Prediction: predict on numbers from a datat set with Timeseries; ex. to predict cypto price in near future

In [63]:
# to make it simpler, I use 2019 data only and split out to do ML, then use 2020 df is for reference 
# after gor the result to check how accurate my prediction method did.

X = train_df.drop(columns=['loan_status'])
X

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,n,29.99,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,n,11.26,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,n,11.28,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,n,18.08,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,n,27.77,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,n,28.42,...,100.0,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N
12176,354944,354944,15000.0,0.1774,540.34,RENT,50000.0,Verified,n,23.43,...,90.5,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N
12177,354973,354973,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,n,28.80,...,100.0,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N
12178,355002,355002,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,n,11.44,...,100.0,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N


In [64]:
y= train_df[['loan_status']]
y

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
12175,high_risk
12176,high_risk
12177,high_risk
12178,high_risk


In [65]:
# check if X has null data:
null_train = train_df.isnull().sum().sum()
null_train

0

In [ ]:
# Optional reference from classmate:
# Convert categorical data to numeric and separate target feature for training data

# for col in train_df.columns:
#     if train_df[col] is not in :
#         train_df[col] = pd.to_numeric(train_df[col], errors='coerce')

# #train_df.dropna(inplace=True)
        
# X = train_df
# X


In [ ]:
#### if we use 2020 df with 'loan_status' column out to be the y; then the missing rows from the 2020 will make splittransform error
####  to prevent from that error; need to make them consistent by: dropna (10% of missing rows) OR:
#### fillna (fill with 0 to all, or fill with the mean of all the other rows, or fill by the 'mode': get the most frequent values show up entire dataset)
##### syntax to do fillna with 0: 
##### for row in X_train.columns:
#####     if row not in X_test.columns:

#####         X_test[row] = 0

In [74]:
X_dummies = pd.get_dummies(X)
X_dummies

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,...,0,1,0,1,1,0,1,0,1,0
1,141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,...,0,1,0,1,1,0,1,0,1,0
2,321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,...,0,1,0,1,1,0,1,0,1,0
3,11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,...,0,1,0,1,1,0,1,0,1,0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,...,0,1,0,1,1,0,1,0,1,0
12176,354944,354944,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,...,1,1,0,1,1,0,1,0,1,0
12177,354973,354973,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,...,0,1,0,1,1,0,1,0,1,0
12178,355002,355002,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,...,0,1,0,1,0,1,1,0,1,0


In [46]:
# Purpose of get_dummies: to convert the string of risk into separate columns to be boolean dtype. 
# 0 = False, 1 = True. 
# y_dummies = pd.get_dummies(y)
# y_dummies

In [66]:
# Converting output labels to 0 and 1; assign the 0 or 1 into the definition of Low risk (1) or high risk (0)
# this step returns only 1 column, this case 
y_label = LabelEncoder().fit_transform(y)
y_label


/Users/ngocdiep/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


array([1, 1, 1, ..., 0, 0, 0])

In [69]:
# X_label = LabelEncoder().fit_transform(X)
# X_label

In [70]:
#this is the code for the machine learning algorithm
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [75]:
# Train the Logistic Regression model on the unscaled data and print the model score
# this is what the machine algotirhtm will train and validate the data it learned.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

In [76]:
classifier.fit(X_train, y_train)

/Users/ngocdiep/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [77]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [78]:
# Train a Random Forest Classifier model and print the model score

clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')


Training Score: 1.0
Testing Score: 0.7944170771756979


In [79]:
# Train the Logistic Regression model on the scaled data and print the model score

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver="lbfgs", random_state=1)
classifier

#Train our model
classifier.fit(X_train_scaled, y_train)
classifier.score(X_test_scaled, y_test)

/Users/ngocdiep/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7155993431855501

In [ ]:
# Note for future reference (tutor)
# after your machine has trained and validated the data on the xtrain, ytrain and xtest and ytets sets are
#and you have found the accuracy of the

#the next step is to see how well your machine (the classifier variable) does on data it has not seen
#this data is coming from the second dataframe
#the classifier will try to predict the loan status based on the x values only from the cleaned second dataframe
#you will then compare the predicted outcome with the actual values of the loan_status from the second dataframe

### Please see more evaluation in my Readme.md file of this repo. Thanks 